# Better match beetween KS slugs and BGG ids

We have notice that many games are not matching between KS and BGG, while the game name slightly change from kickstarter and BGG. 

I want to manually control all possible combination to increase the matching amount of KSS_slugs/BGG_ids

In [226]:
import pandas as pd
import numpy as np
import sql_functions as sf
import Capstone_functions as cp
from IPython.display import clear_output

engine = sf.get_engine()
 
schema = "bgg_data"

## Import tables from SQLpostgres DB

In [227]:
sql = f'''SELECT DISTINCT game_name, slug, date_part('year', created_at),creator_name FROM {schema}.kickstarter_raw AS kr
        WHERE kr.slug NOT IN (SELECT slug 
        FROM {schema}.unique_slug_bgg_id);
      '''
df_ks_unassigned = sf.get_dataframe(sql) # the table contains all unassigned raw KS slugs

In [228]:
sql = f'''SELECT id, game_name, yearpublished FROM (SELECT main.id, game_name, yearpublished FROM {schema}.game_names_masked
      INNER JOIN {schema}.main
      ON main.id = game_names_masked.id) AS ft
      ;'''
df_bgg_names = sf.get_dataframe(sql)

In [229]:
sql = f'''SELECT d.designer_id, designer, id FROM {schema}.designer d
INNER JOIN {schema}.unique_designer ud 
ON d.designer_id = ud.designer_id
      ;'''
df_bgg_design = sf.get_dataframe(sql)

In [230]:
sql = f'''SELECT p.publisher_id, id, publisher_name FROM {schema}.publisher p 
INNER JOIN {schema}.unique_publisher up 
ON p.publisher_id = up.publisher_id
      ;'''
df_bgg_publ = sf.get_dataframe(sql)

## Join and clean different tables to get a KS table and BGG table with year, designer and publisher to compare

In [231]:
df_bgg_names.set_index('id',inplace=True)
df_bgg_design.set_index('id',inplace=True)
df_bgg_publ.set_index('id',inplace=True)

In [232]:
df_bgg_creators = df_bgg_publ.join(df_bgg_design)

In [233]:
df_check_table = df_bgg_creators.join(df_bgg_names)

In [234]:
df_check_table.drop(['publisher_id','designer_id'], axis=1,inplace=True)

In [235]:
df_check_table = df_check_table.reset_index()

In [236]:
df_check_table

,id,publisher_name,designer,game_name,yearpublished
0,1,Hans im Glück,Karl-Heinz Schmiel,die_macher,1986.0
1,1,Hans im Glück,Karl-Heinz Schmiel,德国大选,1986.0
2,1,Hans im Glück,Karl-Heinz Schmiel,디_마허,1986.0
3,1,Moskito Spiele,Karl-Heinz Schmiel,die_macher,1986.0
4,1,Moskito Spiele,Karl-Heinz Schmiel,德国大选,1986.0
...,...,...,...,...,...
1803079,365849,Games Workshop Ltd.,NaN,warhammer_age_of_sigmar_generals_handbook_pitc...,2022.0
1803080,365861,Vacuum Motor Oil Company Pty Ltd,NaN,vac_tour,1939.0
1803081,365871,Button Shy,Jason Tagmire,hush_woven_bog,2022.0
1803082,365871,Button Shy,George Tagmire,hush_woven_bog,2022.0


In [237]:
df_ks_unassigned

,game_name,slug,date_part,creator_name
0,farsight_5e_evolved_explorative_scifi_rpg,farsight-5e-evolved-explorative-sci-fi-rpg,2020.0,Lightfish Games
1,the_essential_guide_to_potions_and_oils,the-essential-guide-to-potions-and-oils,2014.0,Second Thought Games
2,16_games_only_15_per_game,16-games-only-15-per-game,2018.0,Joshua Mason
3,ancient_trails_pandora,ancient-trails-pandora,2015.0,Michael Cross
4,the_ultimate_guide_to_hair,the-ultimate-guide-to-hair,2020.0,Quantum Ogre Publishing
...,...,...,...,...
22022,trayus_stickus_detailed_movement_tray_grip_sur...,trayus-stickus-wargame-movement-tray-grip-surf...,2014.0,Wargames Bakery
22023,book_it_the_wrestling_promoter_card_game,book-it-the-wrestling-promoter-card-game,2018.0,Paul LaPorte
22024,snowhaven_for_fifth_edition,snowhaven-for-fifth-edition,2019.0,Josh Heath
22025,divination_dice_set_reprint,divination-dice-set-re-print,2019.0,Doug Wilson


In [238]:
df_check_table['name_list'] = df_check_table['game_name'].str.split('_')

## Remove not european Titles

In [239]:
df_check_table['game_name'] = df_check_table['game_name'].str.replace(r'[^\x00-\x7F]+', '')

/var/folders/rb/p48bcgy90nn3g9bz4kw5bh4w0000gn/T/ipykernel_3126/997469227.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_check_table['game_name'] = df_check_table['game_name'].str.replace(r'[^\x00-\x7F]+', '')


In [240]:
df_check_table['name_without_underscore'] = df_check_table['game_name'].str.replace('_','')

In [241]:
mask = df_check_table['name_without_underscore'] == ''

df_check_table.loc[mask,'name_without_underscore'] = np.nan

In [242]:
df_check_table.dropna(inplace=True)

In [243]:
df_check_table.drop('name_without_underscore',axis=1,inplace=True)

In [279]:
df_check_table['name_list'] = df_check_table['name_list'].apply(lambda x: [i for i in x if len(i)>3])

In [280]:
df_check_table

,id,publisher_name,designer,game_name,yearpublished,name_list
0,1,Hans im Glück,Karl-Heinz Schmiel,die_macher,1986.0,[macher]
3,1,Moskito Spiele,Karl-Heinz Schmiel,die_macher,1986.0,[macher]
6,1,Ediciones MasQueOca,Karl-Heinz Schmiel,die_macher,1986.0,[macher]
9,1,Portal Games,Karl-Heinz Schmiel,die_macher,1986.0,[macher]
12,1,Spielworxx,Karl-Heinz Schmiel,die_macher,1986.0,[macher]
...,...,...,...,...,...,...
1803076,365846,Wizards of the Coast,Jenna Helland,magic_the_gathering_double_masters_2022,2022.0,"[magic, gathering, double, masters, 2022]"
1803077,365846,Wizards of the Coast,Miguel López,magic_the_gathering_double_masters_2022,2022.0,"[magic, gathering, double, masters, 2022]"
1803081,365871,Button Shy,Jason Tagmire,hush_woven_bog,2022.0,"[hush, woven]"
1803082,365871,Button Shy,George Tagmire,hush_woven_bog,2022.0,"[hush, woven]"


## Try to match KS table

In [247]:
df_ks_unassigned['game_list'] = df_ks_unassigned['game_name'].str.split('_')

In [248]:
df_ks_unassigned

,game_name,slug,date_part,creator_name,game_list
0,farsight_5e_evolved_explorative_scifi_rpg,farsight-5e-evolved-explorative-sci-fi-rpg,2020.0,Lightfish Games,"[farsight, 5e, evolved, explorative, scifi, rpg]"
1,the_essential_guide_to_potions_and_oils,the-essential-guide-to-potions-and-oils,2014.0,Second Thought Games,"[the, essential, guide, to, potions, and, oils]"
2,16_games_only_15_per_game,16-games-only-15-per-game,2018.0,Joshua Mason,"[16, games, only, 15, per, game]"
3,ancient_trails_pandora,ancient-trails-pandora,2015.0,Michael Cross,"[ancient, trails, pandora]"
4,the_ultimate_guide_to_hair,the-ultimate-guide-to-hair,2020.0,Quantum Ogre Publishing,"[the, ultimate, guide, to, hair]"
...,...,...,...,...,...
22022,trayus_stickus_detailed_movement_tray_grip_sur...,trayus-stickus-wargame-movement-tray-grip-surf...,2014.0,Wargames Bakery,"[trayus, stickus, detailed, movement, tray, gr..."
22023,book_it_the_wrestling_promoter_card_game,book-it-the-wrestling-promoter-card-game,2018.0,Paul LaPorte,"[book, it, the, wrestling, promoter, card, game]"
22024,snowhaven_for_fifth_edition,snowhaven-for-fifth-edition,2019.0,Josh Heath,"[snowhaven, for, fifth, edition]"
22025,divination_dice_set_reprint,divination-dice-set-re-print,2019.0,Doug Wilson,"[divination, dice, set, reprint]"


In [277]:
df_ks_unassigned['game_list'] = df_ks_unassigned['game_list'].apply(lambda x: [i for i in x if len(i)>3])

In [278]:
df_ks_unassigned

,game_name,slug,date_part,creator_name,game_list
0,farsight_5e_evolved_explorative_scifi_rpg,farsight-5e-evolved-explorative-sci-fi-rpg,2020.0,Lightfish Games,"[farsight, evolved, explorative, scifi]"
1,the_essential_guide_to_potions_and_oils,the-essential-guide-to-potions-and-oils,2014.0,Second Thought Games,"[essential, guide, potions, oils]"
2,16_games_only_15_per_game,16-games-only-15-per-game,2018.0,Joshua Mason,"[games, only, game]"
3,ancient_trails_pandora,ancient-trails-pandora,2015.0,Michael Cross,"[ancient, trails, pandora]"
4,the_ultimate_guide_to_hair,the-ultimate-guide-to-hair,2020.0,Quantum Ogre Publishing,"[ultimate, guide, hair]"
...,...,...,...,...,...
22022,trayus_stickus_detailed_movement_tray_grip_sur...,trayus-stickus-wargame-movement-tray-grip-surf...,2014.0,Wargames Bakery,"[trayus, stickus, detailed, movement, tray, gr..."
22023,book_it_the_wrestling_promoter_card_game,book-it-the-wrestling-promoter-card-game,2018.0,Paul LaPorte,"[book, wrestling, promoter, card, game]"
22024,snowhaven_for_fifth_edition,snowhaven-for-fifth-edition,2019.0,Josh Heath,"[snowhaven, fifth, edition]"
22025,divination_dice_set_reprint,divination-dice-set-re-print,2019.0,Doug Wilson,"[divination, dice, reprint]"


In [254]:
slug_iter = iter(df_ks_unassigned['game_name'])
choice = ""
cleaning_dict = dict()

while choice != "exit": 
    #mask_check = df_ks_creator_filter["slug"].isin([next(slug_iter)])
    #display(df_ks_creator_filter.loc[mask_check, :].reset_index())
    display(df_check_table)
    choice = input("select the row with the correct BGG ID")
    if choice.isnumeric():
        cleaning_dict[next(slug_iter)] = df_check_table.iat[int(choice),0] 

    clear_output(wait=True)

,id,publisher_name,designer,game_name,yearpublished,name_list
0,1,Hans im Glück,Karl-Heinz Schmiel,die_macher,1986.0,"[die, macher]"
3,1,Moskito Spiele,Karl-Heinz Schmiel,die_macher,1986.0,"[die, macher]"
6,1,Ediciones MasQueOca,Karl-Heinz Schmiel,die_macher,1986.0,"[die, macher]"
9,1,Portal Games,Karl-Heinz Schmiel,die_macher,1986.0,"[die, macher]"
12,1,Spielworxx,Karl-Heinz Schmiel,die_macher,1986.0,"[die, macher]"
...,...,...,...,...,...,...
1803076,365846,Wizards of the Coast,Jenna Helland,magic_the_gathering_double_masters_2022,2022.0,"[magic, the, gathering, double, masters, 2022]"
1803077,365846,Wizards of the Coast,Miguel López,magic_the_gathering_double_masters_2022,2022.0,"[magic, the, gathering, double, masters, 2022]"
1803081,365871,Button Shy,Jason Tagmire,hush_woven_bog,2022.0,"[hush, woven, bog]"
1803082,365871,Button Shy,George Tagmire,hush_woven_bog,2022.0,"[hush, woven, bog]"


In [255]:
cleaning_dict

{'farsight_5e_evolved_explorative_scifi_rpg': 'Ediciones MasQueOca',
 'the_essential_guide_to_potions_and_oils': 'Valley Games, Inc.'}